In [10]:
import os
from aicsimageio import AICSImage
from tifffile import imsave
import numpy as np
from aicssegmentation.core.pre_processing_utils import suggest_normalization_param

# package for 3d visualization
from itkwidgets import view                              
from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 12]

# package for io 
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter

# function for core algorithm
from aicssegmentation.core.vessel import filament_2d_wrapper
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_3d, edge_preserving_smoothing_3d
from skimage.morphology import remove_small_objects     # function for post-processing (size filter)

In [4]:
filename = r"\\allen\aics\assay-dev\MicroscopyOtherData\Viana\datasets\dual_mito\raw\AD00005684_20231208_X02_002.czi - AD00005684_20231208_X02_002.czi #04.tif"
reader = AICSImage(filename) 
IMG = reader.data.astype(np.float32)

print(IMG.shape)

Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 6


(1, 5, 75, 624, 924)


In [20]:
reader

<AICSImage [Reader: TiffReader, Image-is-in-Memory: True]>

In [8]:
#####################
structure_channel = 2

#####################

struct_img0 = IMG[0,structure_channel,:,:,:].copy()

#view(single_fluorescent_view(struct_img0))

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [12]:
# Pre-Processing
################################
## PARAMETERS for this step ##
intensity_scaling_param = [1, 15]
gaussian_smoothing_sigma = 1
################################
# intensity normalization
struct_img = intensity_normalization(struct_img0, scaling_param=intensity_scaling_param)

# smoothing with 2d gaussian filter slice by slice 
structure_img_smooth = image_smoothing_gaussian_3d(struct_img, sigma=gaussian_smoothing_sigma)

In [13]:
suggest_normalization_param(struct_img0)

mean intensity of the stack: 464.6185607910156
the standard deviation of intensity of the stack: 94.28467559814453
0.9999 percentile of the stack intensity is: 1879.11083984375
minimum intensity of the stack: 370.32037353515625
maximum intensity of the stack: 1879.11083984375
suggested upper range is 15.0, which is 1878.8886947631836
suggested lower range is 1.0, which is 370.3338851928711
So, suggested parameter for normalization is [1.0, 15.0]
To further enhance the contrast: You may increase the first value (may loss some dim parts), or decrease the second value(may loss some texture in super bright regions)
To slightly reduce the contrast: You may decrease the first value, or increase the second value


In [14]:
# Core Algorithm
## PARAMETERS for this step ##
f2_param = [[1.5, 0.16]]
################################

bw = filament_2d_wrapper(structure_img_smooth, f2_param)

In [15]:
# Post-Processing
## PARAMETERS for this step ##
minArea = 5
################################

seg = remove_small_objects(bw>0, min_size=minArea, connectivity=1, in_place=False)

In [16]:
seg = seg >0
out=seg.astype(np.uint8)
out[out>0]=255

imsave("SEG_AD00005684_20231208_X02_002.czi - AD00005684_20231208_X02_002.czi #04.tiff", out)

<h3> Batch Processing <h3>

In [29]:
input_folder = "/allen/aics/assay-dev/MicroscopyOtherData/Viana/datasets/dual_mito/raw"
output_folder = "/allen/aics/assay-dev/MicroscopyOtherData/Viana/datasets/dual_mito/segmenter"

for file in os.listdir(input_folder):
    
    # read in and prepare file
    filename_mito = os.path.join(input_folder, file)
    reader = AICSImage(filename_mito) 
    IMG = reader.data.astype(np.float32)
    structure_channel = 2
    struct_img0 = IMG[0,structure_channel,:,:,:].copy()
    
    # pre-processing
    intensity_scaling_param = [1, 15]
    gaussian_smoothing_sigma = 1
    struct_img = intensity_normalization(struct_img0, scaling_param=intensity_scaling_param) # intensity normalization
    structure_img_smooth = image_smoothing_gaussian_3d(struct_img, sigma=gaussian_smoothing_sigma) # smoothing
    
    # Core Algorithm
    f2_param = [[1.5, 0.16]]
    bw = filament_2d_wrapper(structure_img_smooth, f2_param)
    
    # Post-Processing
    minArea = 5
    seg = remove_small_objects(bw>0, min_size=minArea, connectivity=1, in_place=False)
    seg = seg >0
    out=seg.astype(np.uint8)
    out[out>0]=255
    
    #Save
    imsave(output_folder + "/"+ "SEG_" + "{}".format(file), out)
    print("DONE.")  

Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 6
Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 6


DONE.


Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 6


DONE.


Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 6


...

DONE.


Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 6


DONE.


Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 6


DONE.
DONE.
